In [1]:
import pandas as pd
from fuzzywuzzy import process, fuzz
from typing import Dict, Optional, Tuple
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class EventCodeMatcher:
    """Класс для сопоставления мероприятий с их кодами"""
    
    def __init__(self, reference_df: pd.DataFrame, threshold: int = 70):
        self.threshold = threshold
        self.kod_mapping = self._build_mapping(reference_df)
        self.reference_names = list(self.kod_mapping.keys())
        self.match_cache = {}
        
    def _build_mapping(self, df: pd.DataFrame) -> Dict[str, str]:
        """Создает словарь соответствий: нормализованное название -> код"""
        mapping = {}
        for _, row in df.iterrows():
            normalized_name = self._normalize_text(row["Типовое мероприятие"])
            if normalized_name:
                mapping[normalized_name] = row["Код"]
        return mapping
    
    @staticmethod
    def _normalize_text(text) -> str:
        """Нормализация текста для сопоставления"""
        if not isinstance(text, str):
            return ""
        return text.lower().strip()
    
    def _clean_event_name(self, name: str) -> str:
        """Очистка названия мероприятия от суффиксов занятий"""
        normalized = self._normalize_text(name)
        if " - занятие " in normalized:
            return normalized.split(" - занятие ")[0]
        elif " - занятие" in normalized:
            return normalized.split(" - занятие")[0]
        return normalized
    
    def find_best_match(self, event_name: str) -> Optional[str]:
        """Находит наиболее подходящее соответствие для названия мероприятия"""
        if event_name in self.match_cache:
            return self.match_cache[event_name]
        
        cleaned_name = self._clean_event_name(event_name)
        if not cleaned_name:
            return None
        
        matches = process.extract(
            cleaned_name,
            self.reference_names,
            scorer=fuzz.token_sort_ratio,
            limit=1
        )
        
        best_match = matches[0][0] if matches and matches[0][1] >= self.threshold else None
        self.match_cache[event_name] = best_match
        return best_match
    
    def get_code(self, match: str) -> str:
        """Получает код для найденного соответствия"""
        return self.kod_mapping.get(match, "")

def load_data(events_file: str, codes_file: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Загружает данные из файлов"""
    try:
        df_events = pd.read_csv(events_file, encoding='utf-8-sig', sep=';')
        df_codes = pd.read_excel(codes_file)
        
        logger.info(f"Загружено {len(df_events)} мероприятий и {len(df_codes)} кодов")
        return df_events, df_codes
    
    except Exception as e:
        logger.error(f"Ошибка при загрузке данных: {e}")
        raise

def update_event_codes(df_events: pd.DataFrame, matcher: EventCodeMatcher) -> Dict[str, int]:
    """Обновляет коды типовых мероприятий в DataFrame"""
    stats = {"matched": 0, "unmatched": 0, "cleared": 0}
    
    # Используем vectorized операции где возможно
    distance_mask = df_events["Тип мероприятия"] == "Дистанционное мероприятие"
    program_mask = df_events["Тип мероприятия"] == "Программа мероприятий"
    
    # Очищаем коды для дистанционных мероприятий
    df_events.loc[distance_mask, "Код типового мероприятия"] = ""
    stats["cleared"] = distance_mask.sum()
    
    # Обновляем коды для программ мероприятий
    program_events = df_events[program_mask]
    
    for idx in program_events.index:
        event_name = df_events.at[idx, "Название мероприятия"]
        best_match = matcher.find_best_match(event_name)
        
        if best_match:
            df_events.at[idx, "Код типового мероприятия"] = matcher.get_code(best_match)
            stats["matched"] += 1
        else:
            df_events.at[idx, "Код типового мероприятия"] = ""
            stats["unmatched"] += 1
    
    return stats

def save_results(df: pd.DataFrame, output_file: str) -> None:
    """Сохраняет результаты в файл"""
    try:
        df.to_csv(output_file, index=False, encoding='utf-8-sig', sep=';')
        logger.info(f"Результаты сохранены в {output_file}")
    except Exception as e:
        logger.error(f"Ошибка при сохранении: {e}")
        raise

def main():
    """Основная функция"""
    # Конфигурация
    EVENTS_FILE = "1.reordered_file.csv"
    CODES_FILE = "kod_tipovogo.xlsx"
    OUTPUT_FILE = "3.new_code.csv"
    THRESHOLD = 70
    
    try:
        # Загрузка данных
        df_events, df_codes = load_data(EVENTS_FILE, CODES_FILE)
        
        # Создание сопоставителя
        matcher = EventCodeMatcher(df_codes, threshold=THRESHOLD)
        
        # Обновление кодов
        stats = update_event_codes(df_events, matcher)
        
        # Вывод статистики
        logger.info(f"Статистика обработки:")
        logger.info(f"  - Сопоставлено: {stats['matched']}")
        logger.info(f"  - Не найдено соответствий: {stats['unmatched']}")
        logger.info(f"  - Очищено дистанционных: {stats['cleared']}")
        
        # Сохранение результатов
        save_results(df_events, OUTPUT_FILE)
        
    except Exception as e:
        logger.error(f"Критическая ошибка: {e}")
        raise

if __name__ == "__main__":
    main()

2025-07-01 16:31:08,973 - INFO - Загружено 2811 мероприятий и 246 кодов


2025-07-01 16:31:10,094 - INFO - Статистика обработки:


2025-07-01 16:31:10,094 - INFO -   - Сопоставлено: 92


2025-07-01 16:31:10,095 - INFO -   - Не найдено соответствий: 436


2025-07-01 16:31:10,095 - INFO -   - Очищено дистанционных: 2283


2025-07-01 16:31:10,147 - INFO - Результаты сохранены в 3.new_code.csv
